In [1]:
from Data_quality_evaluation import main

f_percent, COM = main.Comprehensive_Evaluation_simple()

d:\software\anaconda\envs\Meta_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


11


Optuna ING: 100%|██████████| 1000/1000 [00:19<00:00, 50.88it/s]


BestValue : 5136.903262774802
{'shift0': -0.031175179888417795, 'shift1': -0.21291762014173368, 'shift2': 0.02263411902517745, 'shift3': -0.08472127728780465, 'shift4': 0.02757707337694479, 'f_shift': 4.7377768397287846e-05}
[[ 7762  7835 29526 ... 29526  7835  7762]
 [ 7835 29526  7307 ...  7307 29526  7835]
 [29526  7307  7708 ...  7708  7307 29526]
 ...
 [29526  7307  7708 ...  7708  7307 29526]
 [ 7835 29526  7307 ...  7307 29526  7835]
 [ 7762  7835 29526 ... 29526  7835  7762]]


In [7]:
print(COM.single)
print(COM.r)
print(COM.N)

4.3333333333333335e-07
0.00024006666666666668
554
